In [4]:
import os
import ase
from ase.io import Trajectory
from ase.visualize import view
import nglview as nv
import ipywidgets
nv.__version__, ipywidgets.__version__
md_dir = "2024-09-18-20-05-36-solvated-gemSmall-DIST-n2n-correctemb"
traj = Trajectory(os.path.join("checkpoints", md_dir, "md.traj"))
nv.show_asetraj(traj)

# TODO: basic stability check on the trajectory (based on bond length deviations)
# How do we compute equilibrium bond lengths?


NGLWidget(max_frame=4148)